# Encuesta continua de hogares 2020

### Cuánto afecta el nivel eductativo y la cantidad de hijos a las posibilidades de inserción laboral de una mujer

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


In [2]:
df = pd.read_spss('HyP_2020_Terceros.sav')

df = df.rename(columns={
    'numero': 'id_hogar',
    'e26': 'sexo',
    'e563': 'genero',
    'e29_6': 'ascendencia',
    'e30': 'parentesco',
    'pobpcoac': 'estado_laboral',
    })

df = df[['id_hogar', 'sexo', 'genero', 'ascendencia', 'parentesco', 'estado_laboral', 'nper']]

In [3]:
df.sample(5)

,id_hogar,sexo,genero,ascendencia,parentesco,estado_laboral,nper
109678,54618,1.0,Varón,Blanca,Otro pariente,"Inactivo, estudiante",7.0
877,10327,2.0,Mujer,Blanca,Hijo/a de ambos,"Inactivo, realiza los quehaceres del hogar",3.0
76316,4147,1.0,Varón,Blanca,Jefe/a de hogar,Ocupados,1.0
141773,50719,2.0,Mujer,Blanca,Jefe/a de hogar,"Inactivo, jubilado",1.0
112589,55735,1.0,Varón,Blanca,Hijo/a solo del esposo/a compañero/a,"Inactivo, estudiante",4.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145166 entries, 0 to 145165
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   id_hogar        145166 non-null  object  
 1   sexo            145166 non-null  float64 
 2   genero          145166 non-null  category
 3   ascendencia     145166 non-null  category
 4   parentesco      145166 non-null  category
 5   estado_laboral  145166 non-null  category
 6   nper            145166 non-null  float64 
dtypes: category(4), float64(2), object(1)
memory usage: 3.9+ MB


In [5]:
df['estado_laboral'].value_counts() / len(df['estado_laboral']) * 100

Ocupados                                      43.915242
Inactivo, jubilado                            16.902029
Menor 14 años                                 15.480898
Inactivo, estudiante                           8.152735
Inactivo, realiza los quehaceres del hogar     5.433779
Inactivo, pensionista                          3.998870
Desocupados propiamente dichos                 3.600705
Inactivo, otro                                 1.193806
Desocupados BT1V                               0.688867
Inactivo, rentista                             0.340989
Desocupados en seguro de desempleo             0.292079
Name: estado_laboral, dtype: float64

## Género

In [6]:
df['genero'].value_counts()

Mujer                    69037
Varón                    57429
0.0                      18571
Sin dato                    78
Mujer trans                 29
Varón trans                 14
Otra                         6
No sabe / No contesta        2
Name: genero, dtype: int64

## Ascendencia

In [7]:
asc_princ_porcentage = df['ascendencia'].value_counts() / len(df['ascendencia']) * 100

asc_princ_porcentage

Blanca                 69.510767
Asiática o amarilla    23.261645
Afro o negra            3.583484
Sin dato                2.384856
Indígena                1.244782
Otra                    0.014466
Name: ascendencia, dtype: float64

## Composición del hogar

In [8]:
df['parentesco'].value_counts() 

Jefe/a de hogar                            58021
Esposo/a o compañero/a                     31354
Hijo/a de ambos                            27083
Hijo/a solo del jefe/a                     15987
Nieto/a                                     4233
Hijo/a solo del esposo/a compañero/a        2288
Padre/madre                                 1601
Hermano/a                                   1070
Otro pariente                                888
Yerno/nuera                                  875
Otro no pariente                             865
Suegro/a                                     412
Sin dato                                     316
Cuñado/a                                     149
Servicio doméstico o familiar del mismo       24
Name: parentesco, dtype: int64

## Habitantes del hogar

In [9]:
familia = pd.pivot_table(df, index='id_hogar', columns='parentesco',values=['nper'], aggfunc='count')

familia['totales'] = familia.apply(lambda x:x.sum(), axis=1)

familia = familia.copy().sort_values(by='totales', ascending=False)

familia

nper                                                   \
parentesco Cuñado/a Esposo/a o compañero/a Hermano/a Hijo/a de ambos   
id_hogar                                                               
56535             0                      1         0              10   
14619             0                      0         0               0   
9287              0                      0         0               0   
23586             0                      0         0               0   
52853             0                      1         0               3   
...             ...                    ...       ...             ...   
37352             0                      0         0               0   
37338             0                      0         0               0   
37337             0                      0         0               0   
37336             0                      0         0               0   
36366             0                      0         0               0   

                                                                        \
parentesco Hijo/a solo del esposo/a compañero/a Hijo/a solo del jefe/a   
id_hogar                                                                 
56535                                         0                      0   
14619                                         0                      4   
9287                                          0                      5   
23586                                         0                      6   
52853                                         0                      0   
...                                         ...                    ...   
37352                                         0                      0   
37338                                         0                      0   
37337                                         0                      0   
37336                                         0                      0   
36366                                         0                      0   

                                                                               \
parentesco Jefe/a de hogar Nieto/a Otro no pariente Otro pariente Padre/madre   
id_hogar                                                                        
56535                    1       4                0             0           0   
14619                    1       4                2             0           0   
9287                     1       6                0             0           0   
23586                    1       3                1             0           0   
52853                    1       5                0             0           0   
...                    ...     ...              ...           ...         ...   
37352                    1       0                0             0           0   
37338                    1       0                0             0           0   
37337                    1       0                0             0           0   
37336                    1       0                0             0           0   
36366                    1       0                0             0           0   

                                                                      \
parentesco Servicio doméstico o familiar del mismo Sin dato Suegro/a   
id_hogar                                                               
56535                                            0        0        0   
14619                                            0        0        0   
9287                                             0        0        0   
23586                                            0        2        0   
52853                                            0        0        0   
...                                            ...      ...      ...   
37352                                            0        0        0   
37338                                            0        0        0   
37337                                     

## Cantidad de habitantes por hogar

In [10]:
familia['totales'].value_counts().sort_index()

1     15749
2     18429
3     11547
4      8634
5      2870
6       887
7       267
8       134
9        44
10       11
11        8
12       10
13        3
16        1
Name: totales, dtype: int64

In [11]:
df['parentesco_num'] = df['parentesco'].cat.codes

df[['parentesco_num','parentesco']].value_counts().sort_index()

parentesco_num  parentesco                             
0               Cuñado/a                                     149
1               Esposo/a o compañero/a                     31354
2               Hermano/a                                   1070
3               Hijo/a de ambos                            27083
4               Hijo/a solo del esposo/a compañero/a        2288
5               Hijo/a solo del jefe/a                     15987
6               Jefe/a de hogar                            58021
7               Nieto/a                                     4233
8               Otro no pariente                             865
9               Otro pariente                                888
10              Padre/madre                                 1601
11              Servicio doméstico o familiar del mismo       24
12              Sin dato                                     316
13              Suegro/a                                     412
14              Yerno/nuera       

## Madres

In [12]:
mask = df['parentesco_num'].isin([3,5,6])
df = df[mask]

df['es_madre'] = np.NaN
df['es_madre'] = df['es_madre'].apply(lambda x:'madre').where(df['parentesco_num'] == 6).where(df['sexo'] == 2)

## Eliminar filas de hombres jefes de hogar

In [13]:
df_madre_y_familia = df.copy().drop(labels=list(df.loc[(df['sexo'] == 1) & (df['parentesco_num'] == 6)].index))

df_madre_y_familia


,id_hogar,sexo,genero,ascendencia,parentesco,estado_laboral,nper,parentesco_num,es_madre
0,100,2.0,Mujer,Blanca,Jefe/a de hogar,Ocupados,1.0,6,madre
2,100,1.0,0.0,Blanca,Hijo/a de ambos,Menor 14 años,3.0,3,NaN
4,1000,2.0,Mujer,Blanca,Hijo/a solo del jefe/a,"Inactivo, pensionista",2.0,5,NaN
11,10001,2.0,0.0,Blanca,Hijo/a de ambos,Menor 14 años,4.0,3,NaN
12,10006,2.0,Mujer,Asiática o amarilla,Jefe/a de hogar,Ocupados,1.0,6,madre
...,...,...,...,...,...,...,...,...,...
145157,9989,2.0,Mujer,Blanca,Jefe/a de hogar,Ocupados,1.0,6,madre
145160,9991,2.0,Mujer,Blanca,Jefe/a de hogar,Ocupados,1.0,6,madre
145161,9992,2.0,Mujer,Blanca,Jefe/a de hogar,Ocupados,1.0,6,madre
145164,9996,2.0,Mujer,Blanca,Jefe/a de hogar,"Inactivo, pensionista",1.0,6,madre


## Composición de familia solo con jefa de hogar

In [14]:
df_familia_madre = df_madre_y_familia[['id_hogar', 'es_madre', 'parentesco', 'parentesco_num']]

df_familia_madre.groupby(by='id_hogar').count()

df_familia_madre.sort_values(by=['parentesco'], ascending=False)

,id_hogar,es_madre,parentesco,parentesco_num
0,100,madre,Jefe/a de hogar,6
94960,48886,madre,Jefe/a de hogar,6
94924,48875,madre,Jefe/a de hogar,6
94926,48876,madre,Jefe/a de hogar,6
94935,48879,madre,Jefe/a de hogar,6
...,...,...,...,...
88623,46530,NaN,Hijo/a de ambos,3
88620,46529,NaN,Hijo/a de ambos,3
88619,46529,NaN,Hijo/a de ambos,3
88616,46528,NaN,Hijo/a de ambos,3


In [15]:
df_madre_y_familia['hijos'] = df_madre_y_familia.groupby('id_hogar')['id_hogar'].transform('count')

df_madre_y_familia['hijos'].value_counts()

2     25056
1     22307
3     15876
4      6068
5      1995
6       738
7       273
8        72
10       10
Name: hijos, dtype: int64

In [16]:
df_madre_y_familia[df_madre_y_familia['es_madre'].notna()].sort_values(by=['hijos'], ascending=False)

,id_hogar,sexo,genero,ascendencia,parentesco,estado_laboral,nper,parentesco_num,es_madre,hijos
9252,13689,2.0,Mujer,Afro o negra,Jefe/a de hogar,Ocupados,1.0,6,madre,8
9303,13704,2.0,Mujer,Afro o negra,Jefe/a de hogar,Ocupados,1.0,6,madre,8
9286,13700,2.0,Mujer,Afro o negra,Jefe/a de hogar,Ocupados,1.0,6,madre,8
9270,13695,2.0,Mujer,Afro o negra,Jefe/a de hogar,Ocupados,1.0,6,madre,8
41898,27477,2.0,Mujer,Blanca,Jefe/a de hogar,"Inactivo, realiza los quehaceres del hogar",1.0,6,madre,8
...,...,...,...,...,...,...,...,...,...,...
3532,11248,2.0,Mujer,Asiática o amarilla,Jefe/a de hogar,"Inactivo, jubilado",1.0,6,madre,1
3523,11245,2.0,Mujer,Asiática o amarilla,Jefe/a de hogar,Ocupados,1.0,6,madre,1
126964,8918,2.0,Mujer,Asiática o amarilla,Jefe/a de hogar,"Inactivo, jubilado",1.0,6,madre,1
127001,8936,2.0,Mujer,Blanca,Jefe/a de hogar,Ocupados,1.0,6,madre,1


In [17]:
df_madre_y_familia['parentesco'].value_counts()

Jefe/a de hogar                            29325
Hijo/a de ambos                            27083
Hijo/a solo del jefe/a                     15987
Cuñado/a                                       0
Esposo/a o compañero/a                         0
Hermano/a                                      0
Hijo/a solo del esposo/a compañero/a           0
Nieto/a                                        0
Otro no pariente                               0
Otro pariente                                  0
Padre/madre                                    0
Servicio doméstico o familiar del mismo        0
Sin dato                                       0
Suegro/a                                       0
Yerno/nuera                                    0
Name: parentesco, dtype: int64

In [18]:
#cantidad de hijos y sueldo
